# Multi Agent Shopping Mall System 

- Author: [Heeah Kim](https://github.com/yellowGangneng)
- Peer Review: 
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/19-Cookbook/16-MultiAgentShoppingMallSystem.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/19-Cookbook/16-MultiAgentShoppingMallSystem.ipynb)

## Overview

안녕하세요, 여러분! 
<br>이번 튜토리얼에서는 **Multi Agent**를 이용한 쇼핑몰 시스템을 구축해 봅시다!
<br>이 튜토리얼이 끝나면 당신은 아주 멋드러진 쇼핑몰의 사장님이 되어있을 겁니다!
<br>아주 든든한 **에이전트**라는 직원을 여러 명 거느린 아주 멋진 사장님 말이죠!

튜토리얼에 본격적으로 들어가기에 앞서, 우리의 쇼핑몰은 어떻게 만들어질 지를 상상해 봅시다!
<br>우선 우리의 쇼핑몰 시스템에 소비자의 입장으로 들어가게 되면,
어떤 물건을 살 것인지 추천을 받고 싶을 것입니다.
<br>추천을 받으면, 물건을 살 수도 있고, 안 살 수도 있겠지요.
<br>만약 산다면, 사용자의 인적 정보를 물어 최종적으로 구매를 확정할 것인지 여부를 질문하게 될 것입니다.
<br>확정을 한다면 해당 아이템의 잔량은 줄어들고, 당신을 위한 물건은 발송이 될 준비가 시작될 것입니다.

여기서 끝이 아닙니다. 만약, 당신의 물건이 마음에 들지 않을 시에는 환불/교환을 요구할 수도 있겠죠!
<br>물론 환불의 세계는 그리 만만하지 않기에, 쇼핑몰 규정에 통과하지 않으면 환불/교환이 반려될 수도 있겠죠.
<br>하지만 우리는 화이트 컨슈머니까요! 분명 정당한 이유가 있기에 환불/교환이 필요할 것입니다.
<br>그러면 당신의 환불/교환 요구는 수리될 것입니다.

당신이 구매한 물건에 대해서, 후기를 쓸 수도 있을 것입니다.
<br>후기를 작성하면, 당신의 후기는 분명 다른 소비자에게 큰 도움이 될 것입니다!

이번엔 우리가 판매자가 되어봅시다! 판매자가 가장 궁금한 건, 본인이 게재한 물건이 얼마나 판매되었는지일 것입니다.
<br>그리고 자신이 판매하고 있는 물건의 잔량을 조정할 수 있겠고, 
<br>새롭게 판매할 물건을 등록할 수도 있을 것입니다.

판매자는 또한, 소비자가 요청한 환불/교환을 승낙할 수도 있고, 거절할 수도 있을 것입니다.
<br>거절을 하면 거절 사유를 사용자에게 반환해 주어야 할 것이고, 승낙을 하면 아쉽지만 판매했던 물건의 잔량이
<br>한 개가 늘어나게 되겠지요.

마지막으로는 서비스 관리자입니다. 
<br>우선, 회원을 관리할 수 있습니다. 판매자, 구매자의 블랙리스트를 두어 서비스를 차단할 수도 있을 것이고,
<br>그리고 판매자에게 게재할 공지사항과 구매자에게 게재할 공지사항을 관리할 수도 있을 것입니다. 
<br>마지막으로 전반적인 쇼핑몰의 운영 상황을 확인 가능한 대시보드를 확인할 수 있을 것입니다.

어떻게 이 볼륨이 상당해 보이는 서비스를 **멀티 에이전트**로 구현할 수 있는 걸까요?
<br>이 튜토리얼과 함께 차근차근 구현해 나가봅시다!

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)

### References

- []()
----

## Environment Setup

Setting up your environment is the first step. See the [Environment Setup](https://wikidocs.net/257836) guide for more details.


**[Note]**

The langchain-opentutorial is a package of easy-to-use environment setup guidance, useful functions and utilities for tutorials.
Check out the  [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [3]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain-anthropic",
        "langchain_community",
        "langchain_text_splitters",
        "langchain_openai",
        "kagglehub"
    ],
    verbose=False,
    upgrade=False,
)


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


You can set API keys in a `.env` file or set them manually.

[Note] If you’re not using the `.env` file, no worries! Just enter the keys directly in the cell below, and you’re good to go.

In [ ]:
from dotenv import load_dotenv
from langchain_opentutorial import set_env

# Attempt to load environment variables from a .env file; if unsuccessful, set them manually.
if not load_dotenv():
    set_env(
        {
            "OPENAI_API_KEY": "",
            "LANGCHAIN_API_KEY": "",
            "LANGCHAIN_TRACING_V2": "true",
            "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
            "LANGCHAIN_PROJECT": "",  # set the project name same as the title
        }
    )

## 데이터를 준비해 봅시다!

우리의 쇼핑몰 데이터를 준비해 봅시다.
우선 기본적으로는 캐글에 있는 `fashion-clothing-products-catalog`을 이용할 것입니다.
해당 데이터셋에는 다음의 정보들이 담겨 있습니다.

- productId
- productName
- productBrand
- Gender
- Price
- Description
- Primary Color

해당 데이터를 받아오는 데에는 두 가지 방법이 있습니다.
1. 직접 사이트에 들어가 데이터를 다운로드 받습니다.
   <br>우리가 데이터를 받아올 사이트는 *kaggle*이라는 데이터 분석 대회 플랫폼으로, 다양한 종류의 데이터셋을 보유하고 있습니다.
   <br>그 중에서도 우리가 사용할 데이터는 하단 URL의 데이터입니다.
   <br>[Fashion Clothing Products Dataset](https://www.kaggle.com/datasets/shivamb/fashion-clothing-products-catalog/data)
   <br>데이터셋에 대한 자세한 설명 및 다양한 인사이트를 얻을만한 자료가 많이 있으니 참고해 보세요!

2. 코드를 이용한 방법입니다. *kaggle*은 파이썬 라이브러리로 *kagglehub*라는 라이브러리를 제공합니다. 해당 라이브러리를 통해서 우리는
   <br>캐글에 존재하는 데이터셋을 아주 손쉽게 다운로드 받을 수 있습니다.

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shivamb/fashion-clothing-products-catalog")

print("Path to dataset files:", path)